In [1]:
from models.AlexNet import AlexNet

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets, transforms
import torchvision
import matplotlib

import os
import numpy as np
from datetime import datetime as dt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
transform = transforms.ToTensor()

batch_size = 600

train_data   = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_data    = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = AlexNet().to(device)
print(model)

AlexNet(
  (layer1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )


In [ ]:
for i, (X_train, y_train) in enumerate(train_data):
    X_train = X_train.to(device)
#    y_train = y_train.to(device)
    break

print(X_train.shape)
    
x = X_train.view(1,3,32,32)
print(x.shape)
x = model.layer1(x)
print(x.shape)
x = model.layer2(x)
print(x.shape)
x = model.layer3(x)
print(x.shape)
x = model.layer4(x)
print(x.shape)
x = model.layer5(x)
print(x.shape)
x = model.flatten(x)
print(x.shape)
x = model.fc(x)
print(x.shape)
x = model.fc1(x)
print(x.shape)
x = model.fc2(x)
print(x.shape)

print(x)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

import time
start_time = time.time()

epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    test_count = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        b+=1
        
        # Apply the model
        y_pred = model(X_train)  # we don't flatten X-train here
        loss = criterion(y_pred, y_train)
 
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print interim results
        if b%10 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{batch_size*b:6}/60000]  loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(batch_size*b):7.3f}%')
        
    train_losses.append(loss)
    train_correct.append(trn_corr)
        
    # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            X_test = X_test.to(device)
            y_test = y_test.to(device)

            # Apply the model
            y_val = model(X_test)

            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()
            test_count += len(predicted)
            
    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
    
    print(f"Loss: {loss.item():10.8f} - Accuracy: {100*tst_corr/test_count:2.2f}")
        
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed  

epoch:  0  batch:   10 [  6000/60000]  loss: 0.38162437  accuracy:  85.967%
epoch:  0  batch:   20 [ 12000/60000]  loss: 0.43402547  accuracy:  85.992%
epoch:  0  batch:   30 [ 18000/60000]  loss: 0.32240543  accuracy:  86.406%
epoch:  0  batch:   40 [ 24000/60000]  loss: 0.35413837  accuracy:  86.675%
epoch:  0  batch:   50 [ 30000/60000]  loss: 0.43505177  accuracy:  86.703%
epoch:  0  batch:   60 [ 36000/60000]  loss: 0.38204834  accuracy:  86.786%
epoch:  0  batch:   70 [ 42000/60000]  loss: 0.33155161  accuracy:  86.976%
epoch:  0  batch:   80 [ 48000/60000]  loss: 0.40643275  accuracy:  87.150%
Loss: 0.63976592 - Accuracy: 81.12
epoch:  1  batch:   10 [  6000/60000]  loss: 0.38860798  accuracy:  89.367%
epoch:  1  batch:   20 [ 12000/60000]  loss: 0.31322128  accuracy:  89.158%
epoch:  1  batch:   30 [ 18000/60000]  loss: 0.26041162  accuracy:  89.417%
epoch:  1  batch:   40 [ 24000/60000]  loss: 0.35341981  accuracy:  89.379%
epoch:  1  batch:   50 [ 30000/60000]  loss: 0.365753